## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-24-20-52-34 +0000,nyt,Israel and the U.S. Withdraw From Talks With H...,https://www.nytimes.com/2025/07/24/us/politics...
1,2025-07-24-20-52-00 +0000,wsj,Stock Market Today: S&P Hits Fourth Straight R...,https://www.wsj.com/livecoverage/stock-market-...
2,2025-07-24-20-51-00 +0000,wsj,Trump Administration Sues New York City Over S...,https://www.wsj.com/us-news/trump-administrati...
3,2025-07-24-20-47-03 +0000,nyt,Here’s what to know.,https://www.nytimes.com/live/2025/07/24/us/tru...
4,2025-07-24-20-46-17 +0000,nypost,Kristin Cabot resigns from Astronomer more tha...,https://nypost.com/2025/07/24/us-news/kristin-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,57
17,new,16
11,trade,16
147,thailand,14
341,russia,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
282,2025-07-24-01-00-00 +0000,wsj,President Trump’s push to introduce a new stan...,https://www.wsj.com/economy/trade/trump-trade-...,134
111,2025-07-24-17-07-10 +0000,nypost,Trump threatens to escalate trade war with new...,https://nypost.com/2025/07/24/business/trump-t...,111
10,2025-07-24-20-38-00 +0000,wsj,President Trump will visit the Federal Reserve...,https://www.wsj.com/economy/central-banking/tr...,109
168,2025-07-24-14-21-00 +0000,wsj,The European Central Bank paused its most aggr...,https://www.wsj.com/economy/central-banking/ec...,103
129,2025-07-24-16-17-00 +0000,wsj,RNC Chair Michael Whatley is expected to annou...,https://www.wsj.com/politics/elections/north-c...,102


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
282,134,2025-07-24-01-00-00 +0000,wsj,President Trump’s push to introduce a new stan...,https://www.wsj.com/economy/trade/trump-trade-...
245,59,2025-07-24-07-33-58 +0000,nypost,Thailand and Cambodia exchange fire in new bor...,https://nypost.com/2025/07/24/world-news/thail...
308,51,2025-07-23-23-17-00 +0000,wsj,Tulsi Gabbard and the White House alleged form...,https://www.wsj.com/politics/policy/tulsi-gabb...
139,48,2025-07-24-15-56-52 +0000,nyt,"MyPillow Founder, Mike Lindell, Will Not Pay o...",https://www.nytimes.com/2025/07/24/us/election...
61,44,2025-07-24-19-15-55 +0000,nypost,"Inside the NYC apartment where woman, 21, was ...",https://nypost.com/2025/07/24/us-news/inside-t...
169,42,2025-07-24-14-19-43 +0000,nypost,Tesla stock plunges as Elon Musk warns company...,https://nypost.com/2025/07/24/business/tesla-s...
109,40,2025-07-24-17-15-51 +0000,bbc,'I can barely do one story. I just feel dizzy'...,https://www.bbc.com/news/articles/crmvmj8kkjno
77,39,2025-07-24-18-33-08 +0000,nyt,Justice Dept. Expected to Interview Epstein As...,https://www.nytimes.com/2025/07/24/us/politics...
250,36,2025-07-24-06-50-43 +0000,nypost,"Plane crashes in Russia’s Far East, killing al...",https://nypost.com/2025/07/24/world-news/russi...
300,36,2025-07-23-23-44-11 +0000,nypost,Kamala Harris’ ex-lover Willie Brown warns her...,https://nypost.com/2025/07/23/us-news/kamala-h...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
